# Combining Base Python and Pandas

The base Python topics we covered in the previous notebook are common in other fields, but in social science most coding uses Pandas features. That said, having a knowledge of the base Python features are useful in combination with Pandas. That is what we cover in this notebook.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We will once again use the NY street data we used for the first activity.

### Exercise

Load that data, using Pandas.

In [ ]:
data = pd.read_csv("../data/Traffic_Volume_Counts.csv")

In [ ]:
borough = pd.read_csv("../data/Traffic_Borough.csv")
borough["borough"] = borough.RBoro.replace({
    1: "Manhattan",
    2: "Bronx",
    3: "Brooklyn",
    4: "Queens",
    5: "Staten Island"
}).astype("category")
borough = borough.dropna(subset=["borough"]).drop_duplicates()
data = data.merge(borough, on="SegmentID", how="left", validate="m:1")

In [ ]:
data

Next, we need to parse our date column. Go ahead and do that now.

In [ ]:
data["Date"] = pd.to_datetime(data.Date, format="%m/%d/%Y")

In [ ]:
data = data.rename(columns={"Roadway Name": "roadway_name"})

## List comprehensions

Let's create a column that has the total flow for the whole day. We could do this by manually specifying every column like this:

```python
data["total"] = data["12:00-1:00AM"] + data["1:00-2:00AM"] + data["2:00-3:00AM"] . . .
```

but this is tedious. Instead, we can use a list comprehension to select all of these columns, and then use the `pandas` `.sum()` function to add them together.

First, we select the columns, and use an assertion to verify that there are 24 of them.

In [ ]:
time_columns = [c for c in data.columns if c.endswith("AM") or c.endswith("PM")]
assert len(time_columns) == 24
time_columns

Next, we sum them all up.

In [ ]:
data["total"] = data[time_columns].sum(axis="columns")
data.total

Now, we can make a histogram. I am increasing the number of bins to get a more granular histogram.

In [ ]:
plt.hist(data.total, bins=50)

## Exercise

Use list comprehensions to create separate `am_total` and `pm_total` columns. Plot both of them. Adjust the bin size of the histogram to your liking.

In [ ]:
data["am_total"] = data[[c for c in data.columns if c.endswith("AM")]].sum(axis="columns")
data["pm_total"] = data[[c for c in data.columns if c.endswith("PM")]].sum(axis="columns")

In [ ]:
plt.hist(data.am_total, bins=50)

In [ ]:
plt.hist(data.pm_total, bins=50)

Many of the streets in this dataset were observed over the course of more than one day. Suppose we are tasked with creating plots that compare daily volumes for several sensors, we might want to average together all the days that a sensor was observed in a particular direction. We can do this with our `time_columns` variable.

In [ ]:
mean_daily_traffic = data.groupby(["SegmentID", "Direction"])[time_columns].mean()
mean_daily_traffic

Now, perhaps we would like to plot the traffic over the course of a day. In introduction to Python, we discussed "long" and "wide" formats. The data we have here are in a "wide" format. To plot it, it will be easier to have it in a "long" format. We can use the `.stack` function to convert it to "long" format. The "stack" function takes the column names and makes them an additional level of the index, with a single column remaining containing the values. `.unstack` is the opposite.

In [ ]:
mean_daily_traffic = mean_daily_traffic.stack()
mean_daily_traffic

This is now a series with a MultiIndex (meaning that the index has multiple columns; we'll talk more about this when we go in-depth talking about indexing in the next exercise). For now, we just want to convert it back to a DataFrame. But notice that the time and count columns do not have names. We can fix this before we convert to a dataframe.

Remember that this is a series, so it is technically only one column; the other three are part of the "index". We can rename the series entirely to represent that it contains counts.

In [ ]:
mean_daily_traffic.rename("mean_count", inplace=True)

For the time column, we need to rename the index level, so we use the `set_names` function of the index. We could do this in place, but I didn't here, to demonstrate how the index works. The index is separate from the data in a Series or DataFrame, so setting the names generates a new index, not a completely new series. Thus, we need to assign it to `mean_daily_traffic.index`

In [ ]:
mean_daily_traffic.index = mean_daily_traffic.index.set_names("time", level=2)
mean_daily_traffic

Now, we can plot this data. Let's plot [sensor 38636, on 125th St in Harlem](https://www.google.com/maps/place/40%C2%B048'38.0%22N+73%C2%B057'07.3%22W/@40.8105409,-73.9565137,17z/data=!3m1!4b1!4m13!1m8!3m7!1s0x89c24ac4ca59bc75:0xb6b907ef32da45c!2sRamona+Ave,+Staten+Island,+NY!3b1!8m2!3d40.5375141!4d-74.2014443!16s%2Fg%2F1v2kybb4!3m3!8m2!3d40.810541!4d-73.952029) in the eastbound and westbound directions. First, we will use indexing to select the east- and westbound data. 

In [ ]:
eb_traffic = mean_daily_traffic.loc[38636, "EB"]
wb_traffic = mean_daily_traffic.loc[38636, "WB"]
wb_traffic

Because this is still a series, as far as matplotlib is concerned it is just a single column of counts. So the y axis is just nb_traffic or sb_traffic; the x axis will be the index. Note that the sensor and direction levels of the index are gone, since we selected based on them.

In [ ]:
plt.plot(eb_traffic.index, eb_traffic, label="Eastbound")
plt.plot(wb_traffic.index, wb_traffic, label="Westbound")
plt.legend()

The labels are hard to read. We can rotate them to make it easier to read.

In [ ]:
plt.plot(eb_traffic.index, eb_traffic, label="Eastbound")
plt.plot(wb_traffic.index, wb_traffic, label="Westbound")
plt.xticks(rotation=45, ha="right")
plt.legend()

## Exercise

How would you describe the traffic pattern here?

Plot the Henry Hudson Parkway between Dyckman St and the Henry Hudson Bridge (segment 132490 southbound and 189377 northbound). This is a major commuter route in and out of New York City. Are the traffic patterns any different?

In [ ]:
nb_traffic = mean_daily_traffic.loc[189377, "NB"]
sb_traffic = mean_daily_traffic.loc[132490, "SB"]

plt.plot(nb_traffic.index, nb_traffic, label="Northbound")
plt.plot(sb_traffic.index, sb_traffic, label="Southbound")
plt.xticks(rotation=45, ha="right")
plt.legend()

## Loops

Loops can be very useful to automate repetitive data analysis tasks. Loops can be used to loop over the rows of a data frame, or can be used to loop over something else, for instance a set of parameters to do sensitivity testing of a model. For instance, we might look at what percentage of the sensors are on roads named "street" rather than road, avenue, drive, etc. We could do this with a loop.

In [ ]:
streets = 0
for row in data.itertuples():
    if "street" in row.roadway_name.lower():
        streets += 1
        
streets /= len(data)
streets *= 100
streets

Loops are also often useful when you want to run an analysis multiple times, but with different parameters - for instance, a sensitivity test of a model. Suppose we want to calculate the percentage of low-volume streets that NYC has put sensors on, but we have several possible definitions for "low-volume" - less than 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, or 10000 vehicles per day.

We could write this out as ten separate calculations, of course, but we can also just put the cutoffs in a list and loop over them and perform the calculation for each.

In [ ]:
# first, compute a mean by segment and direction
by_segment = data.groupby(["SegmentID", "Direction"]).total.mean()

cutoffs = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
for cutoff in cutoffs:
    pct = (by_segment < cutoff).mean() * 100
    print(f"{cutoff}: {pct:.2f}%")

## Forget everything I just told you

Loops are common in most other programming languages, and are common in Python outside of data analytics. However, loops are rare in code using `numpy` or `pandas`. This is because loops in Python carry a high computation overhead, and loops over more than a few thousands items can start to slow down your analysis. Many programmers also find vectorized operations easier to read.

Instead, users are encouraged to use _vectorized operations_, which operate on an entire column/series simultaneously. For instance, we might reimplement the code computing which roads are called "street" using the `str.lower()` and `.str.contains()` function. This will return a boolean (true/false) Series of whether each name contains "street".

In [ ]:
data.roadway_name.str.lower()

In [ ]:
data.roadway_name.str.lower().str.contains("street")

In [ ]:
data.roadway_name.str.lower().str.contains("street").mean() * 100

The previous code using a loop ran plenty fast, but if the dataset was much larger, it might not. The `.str.contains()` format will be much faster in a large dataset, because it uses `pandas` code written in C for the actual operation. In addition, many programmers find the vectorized style more clear - and in this case, I agree.

The loop over different cutoff values is different, because it is looping over a much smaller list - only 10 values. This would be fast even with a large dataset, because the computations use vectorized operations (the vectorized `<`). I would probably write this loop this way. However, some pandas users prefer to avoid loops whenever possible. You could also implement it, by first cutting the `by_segment` series into intervals, and then summing the proportion of the series in each interval. We can use pd.cut and value_counts for this.

pd.cut needs its breakpoints to include the start and end of the data, so first we create a new cutoffs list, adding 0 to the start and infinity to the end. We use the * operator to include all of our existing cutoffs in between. * within a list definition will "unwrap" the list it refers to, adding each element to the new list.

In [ ]:
inclusive_cutoffs = [0, *cutoffs, np.inf]
inclusive_cutoffs

Now, we can use `pd.cut` to divide the data into intervals. `pd.cut` is a function that bins the data into intervals. This is really useful, for instance if you want to group by ranges of a continuous variable.

In [ ]:
interval_data = pd.cut(by_segment, inclusive_cutoffs)
interval_data

Next, we can use `value_counts` to get the count of the number of sensors in each interval. We use `normalize=True` to get a proportion.

`.value_counts()` just returns the count of each unique value in the column it is called on. If normalize is True, it will return proportions rather than raw counts.

In [ ]:
proportions = interval_data.value_counts(normalize=True)
proportions

The last step is to do a cumulative sum of the proportions within each category, and multiply by 100 to get percentages.

In [ ]:
proportions.cumsum() * 100

Does this match what we got before? If it does not, what has gone wrong?

## Exercise

Fix the issue.

In [ ]:
proportions.sort_index().cumsum() * 100

## Exercise

First using a loop and then using vectorized operations, calculate the percentage of observations in `data` that have AM total traffic greater than PM total traffic.

In [ ]:
rows_with_greater_am = 0
for idx, row in data.iterrows():
    if row.am_total > row.pm_total:
        rows_with_greater_am += 1
        
rows_with_greater_am /= len(data)
rows_with_greater_am *= 100
rows_with_greater_am

In [ ]:
(data.am_total > data.pm_total).mean() * 100

Does this accurately represent the proportion of segments that have higher AM traffic? Why or why not?

## Functions

Functions are useful in a wide range of scenarios, because they allow you to encapsulate and re-use the same code in many places. For instance, we could re-write our function that calculates the percentage of segments below a certain cutoff as a function, with the dataset and the cutoff as arguments.

In [ ]:
def percent_below_cutoff(series, cutoff):
    # be careful here that you're referring to the correct dataset. the by_segment
    # data created above is available here as well, so using by_segment instead of series would always
    # refer to that data, even if something else was passed as an argument to the
    # function.
    return (series < cutoff).mean() * 100

In [ ]:
percent_below_cutoff(by_segment, 1000)

We can then run all of our cutoffs fairly easily, by writing them out:

In [ ]:
percent_below_cutoff(by_segment, 2000)

## Exercise

Alternately, we could use a loop like we did before, but use the function to calculate the percentage. Go ahead and do this now.

In [ ]:
for cutoff in cutoffs:
    pct = percent_below_cutoff(by_segment, cutoff)
    print(f"{cutoff}: {pct:.2f}")

We can also easily apply this to a different dataset. For example, maybe we want to apply it to the full dataset instead of segment-level means, so we are calculating the percentage of monitored days, not the percentage of segments.

In [ ]:
percent_below_cutoff(data, 1000)

## Exercise

Why didn't that work?

Hint: what is different between data and by_segment?

Fix the error without modifying the function definition.

In [ ]:
percent_below_cutoff(data.total, 1000)

Functions are extremely useful when analysis gets more complex. For example, suppose we wanted to determine whether traffic is higher during certain parts of the year, but we weren't sure how granular we want the analysis to be. We could write a function that would perform this analysis, with an argument that specifies the granularity, and a conditional to change granularity. This function will take a dataset and a granularity, and will return a count of the total traffic recorded during each period.

In [ ]:
def traffic_totals(dataframe, granularity):
    # Based on the granularity argument, we will define a variable grouping_val that contains the
    # values we wish to group by - months, weeks, etc.
    if granularity == "month":
        grouping_val = dataframe.Date.dt.month
    elif granularity == "week":
        grouping_val = dataframe.Date.dt.weekofyear
    elif granularity == "quarter":
        grouping_val = dataframe.Date.dt.quarter
    else:
        # We haven't seen this before, but this will raise an exception (error) that will
        # be displayed in the notebook if the function is called with something other than
        # month, week, or quarter as the argument.
        raise ValueError(f"Unknown granularity {granularity}")
    
    # adding .sort_index here so they come out in the order they occur in the year
    return dataframe.groupby(grouping_val).total.sum().sort_index()

In [ ]:
monthly_data = traffic_totals(data, "month")

In [ ]:
weekly_data = traffic_totals(data, "week")

We got a warning about Series.dt.weekofyear being _deprecated_, or planned for removal in a future release of Pandas. Go back to the function and modify it based on the suggestion in the warning.

And for completeness, we'll call the function with a granularity we didn't define, to see the error message.

In [ ]:
traffic_totals(data, "day")

## Exercise

Plot the monthly and weekly data on separate plots. Does it appear that there is a time of year that has more traffic?

In [ ]:
plt.plot(monthly_data.index, monthly_data)

In [ ]:
plt.plot(weekly_data.index, weekly_data)

## Exercise

There's a huge spike in October. One possible explanation is that NYC DOT is deploying more counting equipment in October. Write a function that accepts a data frame and a granularity argument and returns a similar result, but with a count of the number of records in each period, rather than the total traffic.

In [ ]:
def count_obs_frequency(dataframe, granularity):
    # Based on the granularity argument, we will define a variable grouping_val that contains the 
    if granularity == "month":
        grouping_val = dataframe.Date.dt.month
    elif granularity == "week":
        grouping_val = dataframe.Date.dt.isocalendar().week
    elif granularity == "quarter":
        grouping_val = dataframe.Date.dt.quarter
    else:
        raise ValueError(f"Unknown granularity {granularity}")
    
    # adding .sort_index here so they come out in the order they occur in the year
    # you couls 
    return dataframe.groupby(grouping_val).size().sort_index()

In [ ]:
monthly_freq = count_obs_frequency(data, "month")
weekly_freq = count_obs_frequency(data, "week")

In [ ]:
plt.plot(monthly_freq.index, monthly_freq)

In [ ]:
plt.plot(weekly_freq.index, weekly_freq)

Yes, it appears the spike is due to additional data collection, not additional traffic.

## Lambda functions

Lambda functions are most often used in conjunction with the `.apply()` function in Pandas. This lets you perform arbitrary operations on each item in a column or each row or column of a DataFrame. For instance, let's adjust the "Roadway Name" field to be in title case. The lambda function will be called once per record.

In [ ]:
data.roadway_name.apply(lambda name: name.title())

Using apply to process all rows of a data frame should be avoided if possible, because it causes the same performance penalties as using a loop. For instance, to capitalize all of the names, it would be more efficient to use `.str.title()`

In [ ]:
data.roadway_name.str.title()

`apply()` can also be used with data frames. This is useful when you need to do some sort of analysis that involves multiple columns. For instance, perhaps we want to create a "description" field that includes the name of the street as well as the cross streets.

When using `apply()` with data frames, you need to pass an `axis` argument, which tells `pandas` whether you want to apply your function to each row or to each column. The syntax of this is confusing - saying `axis="columns"` will run your function once per row. The reasoning is that `axis="columns"` means that every time your function runs, it has all columns for a single row.

In [ ]:
data.apply(lambda row: f"{row.roadway_name} from {row.From} to {row.To}", axis="columns")

As before, this is something that would be quicker without using apply.

In [ ]:
data.roadway_name + " from " + row.From + " to " + row.To

Lastly, we can use apply to apply a function to every column. For example, let's find the interquartile range of the records at each time of day, using our by_segment dataset created above. Pandas does not have a built-in interquartile range function.

Since the function is being applied to each column rather than each row, and there are usually a small number of columns (less than several thousand, anyhow), this is pretty fast.

In [ ]:
data[time_columns].apply(lambda column: column.quantile(0.75) - column.quantile(0.25), axis="rows")

Some programmers would prefer to write this without using apply. You can do this with the quantile function of a dataframe, which also allows you to specify an axis, to compute the 25th and 75th quantiles of each column. This is a case where I actually prefer apply - there's not much performance penalty, and I find it much more readable. But that's just my personal preference.

In [ ]:
data[time_columns].quantile(0.75, axis="rows") - data[time_columns].quantile(0.25, axis="rows")

## Apply with groupby

Probably the place I use `apply` most frequently is with groupby. Apply used in this context runs the function once per group. The usual concerns about overhead apply; this may or may not be an issue, depending on how many groups you have. This is especially useful when you have a calculation that involves multiple columns that you want to run once per group.

Let's look at how well morning traffic predicts evening traffic, by borough. We'll group our data by borough, and then use `apply` to compute the correlation between 8-9 AM and 5-6 PM within each borough.

In [ ]:
data.groupby("borough").apply(lambda group: group["8:00-9:00AM"].corr(group["5:00-6:00PM"]))

How well does morning traffic predict afternoon traffic? Are there any variations by borough?

## Exercise

Use groupby and apply to compute what proportion of segments have higher traffic between 8 and 9 AM than they do between 5 and 6 PM, by borough.

In [ ]:
data.groupby("borough").apply(lambda group: (group["8:00-9:00AM"] > group["5:00-6:00PM"]).mean())